# Calculating NDVI with Landsat and EarthEngine

Initializing the EarthEngine module

In [1]:
from IPython.display import Image

import ee
ee.Initialize()

If the cell above fails, rerun the notebook: hydroshare_earthengine_setup

Calling a Surface Reflectance Landsat image for day 153, 2016, Path 38, Row 31 in World Reference System 2

In [2]:
image = ee.Image('LANDSAT/LC8_SR/LC80380312016153');

Landsat Reflectance product is scaled by 10000

In [3]:
image =image.divide(10000);
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -3.2768001556396484, 'max': 3.276700019836426}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 342285, 0, -30, 4741515]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -3.2768001556396484, 'max': 3.276700019836426}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 342285, 0, -30, 4741515]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -3.2768001556396484, 'max': 3.276700019836426}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 342285, 0, -30, 4741515]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -3.2768001556396484, 'max': 3.276700019836426}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 342285, 0, -30, 4741515]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'

Display the image in RGB color.

In [4]:
imgParams = {'bands': 'B4,B3,B2','min': 0.0,'max': 0.5,'gamma': 1.5}
url = image.getThumbURL(imgParams)
Image(url=url) 

EEException: Total request size (185375163 bytes) must be less than or equal to 16777216 bytes.

Display the image in false color (NIR G B)

In [ ]:
imgParams = {'bands': 'B5,B4,B3','min': 0.0,'max': 0.5,'gamma': 1};
url = image.getThumbURL(imgParams)
Image(url=url) 

Calculating NDVI 

In [ ]:
ndvi = image.normalizedDifference(['B5','B4']);

palette = ['00008b', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301'];

ndvi=ndvi.visualize(min=0,max=1,palette= palette)
Image(url=ndvi.getThumbURL()) 

Zooming into Logan, UT

In [ ]:
geometry = ee.Geometry.Point([-111.84356689453125, 41.737855982210085])
Logan=geometry.buffer(5000) #meters

In [ ]:
ndvi_logan = ndvi.clip(Logan)

In [ ]:
Image(url=ndvi_logan.getThumbURL()) 

In [ ]:
import ipyleaflet
from ipyleaflet import *

m = ipyleaflet.Map(center=(41.7370,-111.8338),zoom=8)


In [ ]:
# adding some_file.py
import sys
sys.path.insert(0, '/home/jovyan/work/notebooks/prj_earth_engine_hydroshare/eeus2017-python')

import ipyleafletee

In [ ]:
# adding NDVI to the dynamic map
map_id = ndvi.getMapId()

ndvi_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
ndvi_lyr = ipyleafletee.TileLayer(url=ndvi_tile_url)
m.add_layer(ndvi_lyr)


In [ ]:
#adding image to the dynamic map
map_id = image.visualize(bands='B5,B4,B3',min=0.0,max=0.7,gamma=1.5).getMapId()

img_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
img_lyr = ipyleafletee.TileLayer(url=img_tile_url)
m.add_layer(img_lyr)


In [ ]:
#m = ipyleafletee.Map(center=(41.7370,-111.8338),zoom=8)
#control = SplitMapControl(left_layer=img_lyr, right_layer=ndvi_lyr)

m.add_control(ipyleaflet.LayersControl())
m